In [1]:
import sys
sys.path.append('../scripts')

In [2]:
import pandas as pd
import spacy
import os
from pathlib import Path
from xml.dom import minidom
from charsplit import Splitter
from rules.ellipticCompounds import exchangeLemmas
from evaluation import error_analysis, get_scores

nlp = spacy.load("de_core_news_lg")

In [3]:
import hydra
from hydra import compose, initialize

hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path=Path('..'), job_name='foo', version_base='1.1')
config = compose(config_name='experiment.yaml')

In [4]:
tmp_path = Path('../tmp/baseline_files')
tmp_path.mkdir(exist_ok=True, parents=True)

In [5]:
path_xml_file = tmp_path / 'Original.xml'
path_output_file = tmp_path / 'Output.xml'
path_gertwol_file = tmp_path / 'gertwol.txt'
path_word_freqs = tmp_path / 'wordFreqs.txt'
tagged_path = '../compoundListTagged.txt'

path_tokens = Path('..') / config.data.ggponc_plain_text

# Load the Data

In [6]:
ellipses = pd.read_csv(Path('..') / config.data.cnf_tsv_path, sep='\t')
not_ellipses = pd.read_csv(Path('..') / config.data.controls_tsv_path, sep='\t')

not_ellipses['full_resolution'] = not_ellipses.raw_sentence
#ellipses = ellipses[ellipses.split == 'test']
#not_ellipses = not_ellipses[not_ellipses.split == 'test']

# Train and Test split?

data = pd.concat([ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']], not_ellipses[['file', 'sentence_id', 'raw_sentence', 'full_resolution']]]).sort_values(by='file').reset_index(drop=True)

In [9]:
data.shape

(6536, 4)

In [10]:
data.head()

,file,sentence_id,raw_sentence,full_resolution
0,00_mundhoehlenkarzinom_0002.tsv,1,Hauptrisikofaktoren für das Auftreten eines Mu...,Hauptrisikofaktoren für das Auftreten eines Mu...
1,00_mundhoehlenkarzinom_0002.tsv,2,Bei chronischem Tabak- oder Alkoholabusus ist ...,Bei chronischem Tabakabusus oder Alkoholabusus...
2,00_mundhoehlenkarzinom_0005.tsv,12,Die hohe Diskrepanz zwischen p16 Immunhistoche...,Die hohe Diskrepanz zwischen p16 Immunhistoche...
3,00_mundhoehlenkarzinom_0005.tsv,18,Die Prävalenz von HPV16 E6 oder E7 Antikörpern...,Die Prävalenz von HPV16 E6 Antikörpern oder HP...
4,00_mundhoehlenkarzinom_0005.tsv,1,Findet man eine vornehmlich weiße Läsion der M...,Findet man eine vornehmlich weiße Läsion der M...


# Create XML Document

In [11]:
root = minidom.Document()

xml = root.createElement('corpus')
xml.setAttribute('id', 'corpus')
root.appendChild(xml)

for i, f in enumerate(data[['file', 'raw_sentence']].groupby('file').first().index.to_list()):
    file = root.createElement('file')
    file.setAttribute('n', str(i))
    xml.appendChild(file)

    div = root.createElement('div')
    file.appendChild(div)

    for j, s in enumerate(data[data['file']==f]['raw_sentence']):
        sentence = root.createElement('s')
        sentence.setAttribute('lang', 'de')
        sentence.setAttribute('n', f'{i}-{j}')
        div.appendChild(sentence)

        doc = nlp(s)
        for k, token in enumerate(doc):
            word = root.createElement('w')
            word.setAttribute('lemma', token.lemma_)
            word.setAttribute('n', f'{i}-{j}-{k}')
            word.setAttribute('pos', token.tag_)
            word.setAttribute('whitespace', token.whitespace_)
            sentence.appendChild(word)
            word.appendChild(root.createTextNode(token.text))

with open(path_xml_file, "wb") as f:
    f.write(root.toprettyxml(indent ="\t", encoding="utf-8"))

# Produce Gertwol-like List and Gather Word Frequencies

In [12]:
freq = {}
dict = {}
splitter = Splitter()
special_characters = "!@#$%^&*()-+?_=,<>/.[] "

for filename in os.listdir(path_tokens):
    with open(path_tokens / filename , encoding='utf8') as f:
        for word in f.readlines():
            word = word[:-1]

            # Gertwol Split
            split = splitter.split_compound(word)[0]
            if split[0] != 0:
                dict[word] = '#'.join([split[1]] + [subword.lower() for subword in split[2:]])

            # Word Freq
            if word in special_characters:
                continue
            if word in freq:
                freq[word] += 1
            else:
                freq[word] = 1

with open(path_gertwol_file, 'w', encoding='utf-8') as f:
    for key, value in dict.items():
        f.write(f'{key}\t{value}\n')

with open(path_word_freqs, 'w', encoding='utf-8') as f:
    for key, value in freq.items():
        f.write(f'{key}\t{value}\n')

# Run Aepli Baseline

In [13]:
exchangeLemmas(path_xml_file, path_output_file, path_gertwol_file, path_word_freqs, tagged_path)

Alkohol#abusus
Alkohol#abusus
Hals#tumoren
Funktions#erhalt
Nährstoff#aufnahme
Schluck#funktion
Nikotin#assoziierte
Kar#zinome
Nikotin#assoziierte
Kar#zinome
Innenspangenre#sektion
Schluck#funktion
Schluck#funktion
Knochen#transplantation
Knochen#transplantation
Mund#bodenkarzinom
Chemotherapie#bedingte
Chemotherapie#bedingte
Radio#chemotherapie
Mund#bodenkarzinoms
Nerv#infiltration
Radio#chemotherapie
Radio#chemotherapie
Mund#pflege
ggf.
Geschmacks#störungen
Geschmacks#störungen
Radio#chemotherapie
Radio#chemotherapie
Radio#chemotherapie
Radio#chemotherapie
Leistungs#zustand
Immun#zellen
Immun#zellen
Gesamt#überleben
Leistungs#zustand
Leistungs#zustand
Konvention#ell
Radio#chemotherapie
Schluck#störungen
Schluck#störungen
Schluck#rehabilitation
Schluck#störung
Schluck#störung
Behandlungs#bedingten
Beeinträchtig#ungen
Sprech#vermögen
Ref#lux
Umwelt#faktoren
Nicht#kardiakarzinom
Tonsillen#karzinome
Peutz-Jeghers#syndrom
Eier#stockkrebs
Familien#anamnese
Missense#mutationen
Familien#anam

# Reconstruct Complete Sentences

In [14]:
dom = minidom.parse(open(path_output_file))
predictions = []

sentences = dom.getElementsByTagName('s')
for sentence in sentences:
    tokens = sentence.getElementsByTagName('w')
    sentence_tokens = []
    for token in tokens:
        word = token.childNodes[0].nodeValue
        lemma = token.getAttribute('lemma')
        if '+' in lemma:
            word_parts = lemma.partition('+')
            if token.getAttribute('pos') == 'TRUNC':
                word = word.replace('-', word_parts[2])
            else:
                word = word.replace('-', word_parts[0])
        whitespace = token.getAttribute('whitespace')
        sentence_tokens.append(word + whitespace)

    predictions.append(''.join(token for token in sentence_tokens))

# Evaluate Baseline

In [15]:
ea = error_analysis(predictions, data.full_resolution, data.raw_sentence)
get_scores(ea, "eval")

{'eval/tp': 0.653610771113831,
 'eval/tp_abs': 4272,
 'eval/fn': 0.23271113831089352,
 'eval/fn_abs': 1521,
 'eval/fp': 0.008567931456548347,
 'eval/fp_abs': 56,
 'eval/replace': 0.0377906976744186,
 'eval/replace_abs': 247,
 'eval/insert': 0.0151468788249694,
 'eval/insert_abs': 99,
 'eval/delete': 0.030599755201958383,
 'eval/delete_abs': 200,
 'eval/complex': 0.02157282741738066,
 'eval/complex_abs': 141,
 'eval/edit_distance_rel': 0.7271554699792715,
 'eval/exact_match': 0.653610771113831,
 'eval/gleu': 0.9587613619221429,
 'eval/edit_distance_abs': 3.2481640146878825}

In [16]:
from notebook_util import show_errors

In [17]:
errors = pd.concat([ea, data[['file', 'sentence_id']].reset_index()], axis=1)

In [24]:
errors[errors.error_type == 'fp'][['file', 'sentence_id', 'pred']].to_excel('../potential_missing.xlsx')

In [21]:
show_errors(errors[errors.error_type == 'fp'])

0;01_magenkarzinom_0245.tsv;89

__Input:__

Der Einsatz neuer endoskopischer Verfahren (NBI, Chromoendoskopie, konfokale Lasermikroskopie) über eine Videoendoskopie hinaus in der Primärdiagnostik von Ösophagus- und Magenkarzinom ist routinemäßig nicht notwendig.

__Error type:__ fp

1;03_lungenkarzinom_0209.tsv;23

...

__Error type:__ fp

55;28_komplementaermedizin_0277.tsv;36

__Input:__

Es liegen Tier- sowie klinische Studien vor, welche erste Hinweise darauf geben, dass Coriolus zur Vorbeugung und Behandlung von Krebs, zur Reduktion von Nebenwirkungen während der Chemotherapie und Strahlentherapie, zur Stimulation des Immunsystems, zur Behandlung von Infektionen, Herpes und Hepatitis, sowie zur Verbesserung von Kraft und Ausdauer angewendet werden könnte.

__Error type:__ fp